In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuração de estilo para os gráficos
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

try:
    df = pd.read_csv("/funcionarios_empresa.csv")
    print("Dataset carregado com sucesso!\n")
except FileNotFoundError:
    print("Erro: O arquivo 'funcionarios_empresa.csv' não foi encontrado.")
    # Encerrar se o arquivo não for encontrado
    exit()

Dataset carregado com sucesso!



In [2]:
print("## Análise Exploratória Inicial (EDA)\n")

# Primeiras 5 linhas
print("### Primeiras 5 linhas do DataFrame:\n")
print(df.head().to_markdown(index=False, numalign="left", stralign="left"))
print("\n" + "="*80 + "\n")

# Informações gerais (tipos de dados e contagem de não-nulos)
print("### Informações e Tipos de Dados:\n")
df.info()
print("\n" + "="*80 + "\n")

# Estatísticas descritivas para colunas numéricas
print("### Estatísticas Descritivas para Colunas Numéricas:\n")
print(df.describe().round(2).to_markdown(numalign="right", stralign="right"))
print("\n" + "="*80 + "\n")

# Contagem de valores únicos para colunas categóricas importantes
print("### Contagem de Valores Únicos (Top 5):\n")
for col in ['Departamento', 'Nivel_Educacao', 'Estado_Civil', 'Tipo_Contrato']:
    print(f"**{col}**:")
    print(df[col].value_counts().head().to_markdown(numalign="left", stralign="left"))
    print("-" * 20)
print("\n" + "="*80 + "\n")

## Análise Exploratória Inicial (EDA)

### Primeiras 5 linhas do DataFrame:

| ID_Funcionario   | Nome               | Idade   | Departamento   | Cargo                    | Salario   | Tempo_Empresa_Meses   | Data_Admissao   | Nivel_Educacao    | Estado_Civil   | Tipo_Contrato   | Avaliacao_Performance   | Horas_Extras_Mes   | Beneficios   | Faltas_Ultimo_Ano   | Satisfacao_Trabalho   |
|:-----------------|:-------------------|:--------|:---------------|:-------------------------|:----------|:----------------------|:----------------|:------------------|:---------------|:----------------|:------------------------|:-------------------|:-------------|:--------------------|:----------------------|
| 1                | Tatiane Mendes 001 | 40      | Vendas         | Vendedor Jr              | 3685.36   | 4                     | 2025-05-18      | Superior Completo | Casado(a)      | CLT             | 4.8                     | 5                  | 953.49       | 1                   | 5.6     

In [3]:
print("## Identificação e Tratamento de Valores Nulos\n")

# Identificar valores nulos
print("### Contagem de Valores Nulos por Coluna:\n")
null_counts = df.isnull().sum()
print(null_counts[null_counts > 0].sort_values(ascending=False).to_markdown(numalign="left", stralign="left"))
print("\n" + "="*80 + "\n")

# Exemplo de Tratamento (Se houver nulos, usamos a mediana/moda como exemplo)
# Para este dataset específico, parece que não há nulos nas amostras,
# mas se houvesse, o tratamento seria:

if null_counts.sum() > 0:
    print("### Estratégia de Tratamento de Nulos:\n")
    # Exemplo: Preencher nulos em 'Salario' com a mediana
    if 'Salario' in df.columns and df['Salario'].isnull().any():
        mediana_salario = df['Salario'].median()
        df['Salario'].fillna(mediana_salario, inplace=True)
        print(f"Valores nulos em 'Salario' preenchidos com a mediana ({mediana_salario:.2f}).")

    # Exemplo: Preencher nulos em 'Nivel_Educacao' com a moda
    if 'Nivel_Educacao' in df.columns and df['Nivel_Educacao'].isnull().any():
        moda_educacao = df['Nivel_Educacao'].mode()[0]
        df['Nivel_Educacao'].fillna(moda_educacao, inplace=True)
        print(f"Valores nulos em 'Nivel_Educacao' preenchidos com a moda ('{moda_educacao}').")
else:
    print("Não foram encontrados valores nulos no dataset.")

print("\n" + "="*80 + "\n")

## Identificação e Tratamento de Valores Nulos

### Contagem de Valores Nulos por Coluna:

|                       | 0   |
|:----------------------|:----|
| Avaliacao_Performance | 25  |
| Satisfacao_Trabalho   | 15  |
| Beneficios            | 10  |
| Nivel_Educacao        | 5   |


### Estratégia de Tratamento de Nulos:

Valores nulos em 'Nivel_Educacao' preenchidos com a moda ('Superior Completo').




/tmp/ipython-input-2042310858.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Nivel_Educacao'].fillna(moda_educacao, inplace=True)


In [ ]:
print("## 🚨 Detecção e Análise de Outliers\n")

# Colunas numéricas para análise de outliers
numerical_cols = ['Salario', 'Idade', 'Tempo_Empresa_Meses', 'Avaliacao_Performance', 'Horas_Extras_Mes', 'Beneficios', 'Faltas_Ultimo_Ano', 'Satisfacao_Trabalho']

# Plotagem de Boxplots
print("### Boxplots para Colunas Numéricas:\n")
plt.figure(figsize=(15, 10))
for i, col in enumerate(numerical_cols):
    plt.subplot(3, 3, i + 1)
    sns.boxplot(y=df[col], color='skyblue')
    plt.title(f'Boxplot de {col}', fontsize=12)
    plt.ylabel(col)
plt.tight_layout()
plt.show()

# Exemplo de Detecção de Outliers usando o método IQR (Intervalo Interquartil)
print("\n### Detecção de Outliers com IQR (para Salário):\n")

coluna_outlier = 'Salario'

Q1 = df[coluna_outlier].quantile(0.25)
Q3 = df[coluna_outlier].quantile(0.75)
IQR = Q3 - Q1

# Limites para detecção
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# Identificar outliers
outliers = df[(df[coluna_outlier] < limite_inferior) | (df[coluna_outlier] > limite_superior)]

print(f"Q1 de '{coluna_outlier}': {Q1:.2f}")
print(f"Q3 de '{coluna_outlier}': {Q3:.2f}")
print(f"IQR de '{coluna_outlier}': {IQR:.2f}")
print(f"Limite Inferior (Outlier): {limite_inferior:.2f}")
print(f"Limite Superior (Outlier): {limite_superior:.2f}")
print(f"\nTotal de Outliers detectados na coluna '{coluna_outlier}': **{len(outliers)}**")

if len(outliers) > 0:
    print("\nExemplo de Outliers (Apenas ID, Salário e Cargo):\n")
    # Mostrar os outliers identificados
    print(outliers[['ID_Funcionario', 'Nome', 'Cargo', 'Salario']].head().to_markdown(index=False, numalign="right", stralign="left"))
else:
    print(f"\n Não foram detectados outliers significativos na coluna '{coluna_outlier}' usando 1.5 * IQR.")

# Exemplo de Tratamento de Outliers (Substituir Outliers pela mediana)
print("\n### Exemplo de Tratamento (Substituição pela Mediana):\n")
if len(outliers) > 0:
    mediana_salario = df[coluna_outlier].median()
    # Cria uma cópia do DataFrame para o tratamento (boas práticas)
    df_tratado = df.copy()

    # Substitui os valores de Salário que são outliers pela mediana
    df_tratado[coluna_outlier] = np.where(
        (df_tratado[coluna_outlier] < limite_inferior) | (df_tratado[coluna_outlier] > limite_superior),
        mediana_salario,
        df_tratado[coluna_outlier]
    )

    print(f"Total de Outliers em '{coluna_outlier}' substituídos pela mediana ({mediana_salario:.2f}).")
    print(f"O novo desvio padrão de '{coluna_outlier}' (tratado) é: {df_tratado[coluna_outlier].std():.2f}")
    print(f"O desvio padrão original era: {df[coluna_outlier].std():.2f}")
else:
    print("Não há outliers detectados para tratamento com 1.5 * IQR.")

print("\n" + "="*80 + "\n")
print("Análise Exploratória e Tratamento concluídos.")

# Plotar o boxplot após o tratamento de Salário (se houve tratamento)
if 'df_tratado' in locals() and len(outliers) > 0:
    plt.figure(figsize=(6, 4))
    sns.boxplot(y=df_tratado[coluna_outlier], color='lightcoral')
    plt.title(f'Boxplot de {coluna_outlier} (Após Tratamento)', fontsize=12)
    plt.ylabel(coluna_outlier)
    plt.show()

## 🚨 Detecção e Análise de Outliers

### Boxplots para Colunas Numéricas:

